# Creating Neural networks using perceptrons

## Create a perceptron class for logic gates

In [ ]:
import numpy as np

class Perceptron(object):
    """Perceptron classifier.
        Parameters
        ------------
        eta : Learning rate
        n_iter : Number of passes over the training dataset.
        Attributes
        -----------
        w_ : 1d-array, weights after fitting.
        errors_ : list, number of misclassifications in every epoch.
    """

    def __init__(self, eta=0.01, n_iter=10, y_values=(0, 1)):
        #Class constructor
        self.eta = eta
        self.n_iter = n_iter
        self.y_values = y_values

    def fit(self, X, y):
        """Fit training data.
            Parameters
            ----------
            X : {array-like}, shape = [n_samples, n_features], Training vectors, where n_samples is the 
                        number of samples and n_features is the number of features.
            y : array-like, shape = [n_samples], Target values.
            Returns
            -------
            self : object
        """
        #X.shape is tuple (20,2), X.shape[1] will return 2
        #w_ will be initialized to an array of 1+2=3 zeros, ie: array([0., 0., 0.])
        self.w_ = np.zeros(1 + X.shape[1])
        #initialize errors_ to blank list
        self.errors_ = []
        #for _ in .... is a way to do the looping but without caring about the number in loop
        #range function creates a tuple with first value as 0 and n_iter
        for _ in range(self.n_iter):
            errors = 0
            # zip function can be used to join two lists together, element wise.
            # So zip of [[0,0], [0,1]] and [0,1] will give [0, 0] and 0 in first loop, and [0, 1]
            # and 1 in second loop
            for xi, target in zip(X, y):
                #eta will be multiplied by error to find the next update
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
            print("weights: {}, rate: {}, iteration: {}, errors: {}".format(self.w_, \
                                                                            self.eta, \
                                                                            self.n_iter, \
                                                                            self.errors_))
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(
            self.net_input(X) >= 0.0, self.y_values[1], self.y_values[0])

### Create AND gate using perceptron

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,0,0,1])

In [ ]:
y = np.array([0,0,0,1])

In [ ]:
and_ppt = Perceptron(eta=1, n_iter=10, y_values=(0, 1))

In [ ]:
and_ppt = and_ppt.fit(X,y)

In [ ]:
print(and_ppt.predict([0,0]))
print(and_ppt.predict([0,1]))
print(and_ppt.predict([1,0]))
print(and_ppt.predict([1,1]))

### Create OR Gate

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,1])

or_ppt = Perceptron(eta=1,n_iter=10)
or_ppt = or_ppt.fit(X,y)

In [ ]:
print(or_ppt.predict([0,0]))
print(or_ppt.predict([0,1]))
print(or_ppt.predict([1,0]))
print(or_ppt.predict([1,1]))

### Create XOR gate using above perceptron, will fail
XOR gate is special in sense its not a linear function and thus the perceptron will fail to converge

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])

xor_ppt = Perceptron(eta=-0.5,n_iter=20)
xor_ppt = xor_ppt.fit(X,y)

In [ ]:
print(xor_ppt.predict([0,0]))
print(xor_ppt.predict([0,1]))
print(xor_ppt.predict([1,0]))
print(xor_ppt.predict([1,1]))

## XOR solution using DNN and new activation functions
XOR cannot be solved using a simple one layer perceptron. Reason for this is XOR is not a **Linearly Separable** function. It needs one input layer, one hidden layer and one output layer.<br>
Tanh is better than sigmoid.
![NueralNetActivationFunctions](./img/nnet-error-functions.png)
For more information please see this
__[link](https://medium.com/@jayeshbahire/the-xor-problem-in-neural-networks-50006411840b)__


In [ ]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1.0 - x**2

class NeuralNetwork:
    def __init__(self, layers, activation="tanh", epochs=1000, learning_rate=0.2):
        if activation == "sigmoid":
            self.activation = sigmoid
            self.activation_derivative = sigmoid_derivative
        elif activation == "tanh":
            self.activation = tanh
            self.activation_derivative = tanh_derivative

        # Set weights
        self.weights = []
        # layers = [2,2,1]
        # range of weight values (-1,1)
        # input and hidden layers - random((2+1, 2+1)) : 3 x 3
        for i in range(1, len(layers) - 1):
            r = 2 * np.random.random((layers[i - 1] + 1, layers[i] + 1)) - 1
            self.weights.append(r)
        # output layer - random((2+1, 1)) : 3 x 1
        r = 2 * np.random.random((layers[i] + 1, layers[i + 1])) - 1
        self.weights.append(r)
        self.epochs=epochs
        self.learning_rate=learning_rate

    def fit(self, X, y):
        # Add column of ones to X
        # This is to add the bias unit to the input layer.
        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)

        for k in range(self.epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                dot_value = np.dot(a[l], self.weights[l])
                activation = self.activation(dot_value)
                a.append(activation)
            # output layer
            error = y[i] - a[-1]
            deltas = [error * self.activation_derivative(a[-1])]

            # we need to begin at the second to last layer
            # (a layer before the output layer)
            for l in range(len(a) - 2, 0, -1):
                deltas.append(deltas[-1].dot(self.weights[l].T) *
                              self.activation_derivative(a[l]))

            # reverse
            # [level3(output)->level2(hidden)]  => [level2(hidden)->level3(output)]
            deltas.reverse()

            # backpropagation
            # 1. Multiply its output delta and input activation
            #    to get the gradient of the weight.
            # 2. Subtract a ratio (percentage) of the gradient from the weight.
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += self.learning_rate * layer.T.dot(delta)

            if k % (self.epochs/10) == 0: print('epochs:{0}'.format(k))

    def predict(self, x):
        a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        return np.where(abs(a) >= 0.5, 1, 0)
        #return a

### Sigmoid function also fails to converge for the data.

In [ ]:
nn = NeuralNetwork([2, 2, 1],
                   activation="sigmoid",
                   epochs=10000,
                   learning_rate=0.2)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
nn.fit(X, y)
for e in X:
    print(e, nn.predict(e))

### Tanh successfully converges for XOR function even in 10000 epochs

In [ ]:
nn = NeuralNetwork([2, 2, 1],
                   activation="tanh",
                   epochs=10000,
                   learning_rate=0.2)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
nn.fit(X, y)
for e in X:
    print(e, nn.predict(e))

# IRIS Data processing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', -1, 1)
X = df.iloc[0:100, [0, 2]].values
plt.scatter(X[:50, 0], X[:50, 1], color='red', marker='o', label='setosa')
plt.scatter(X[50:100, 0], X[50:100, 1], color='blue', marker='x', label='versicolor')
plt.xlabel('sepal length')
plt.ylabel('petal length')
plt.legend(loc='upper left')
plt.show()

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10,y_values=(-1,1))
ppn.fit(X, y)
plt.plot(range(1, len(ppn.errors_) + 1), ppn.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of misclassifications')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
def plot_decision_regions(X, y, classifier, resolution=0.02):
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                         np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.legend(loc='upper left')
plt.show()

In [ ]:
import os
import urllib.request

import numpy as np
import tensorflow as tf

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

def main():
    if not os.path.exists(IRIS_TRAINING):
        raw = urllib.request.urlopen(IRIS_TRAINING_URL).read()
        with open(IRIS_TRAINING, "wb") as f:
            f.write(raw)

    if not os.path.exists(IRIS_TEST):
        raw = urllib.request.urlopen(IRIS_TEST_URL).read()
        with open(IRIS_TEST, "wb") as f:
            f.write(raw)
# Load datasets.
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
                                                                       filename=IRIS_TRAINING,
                                                                       target_dtype=np.int,
                                                                       features_dtype=np.float32)
    test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
                                                                   filename=IRIS_TEST,
                                                                   target_dtype=np.int,
                                                                   features_dtype=np.float32)
# Specify that all features have real-value data
    feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]
# Build 3 layer DNN with 10, 20, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")
# Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
                                                        x={"x": np.array(training_set.data)},
                                                        y=np.array(training_set.target),
                                                        num_epochs=None,
                                                        shuffle=True)
# Train model.
    classifier.train(input_fn=train_input_fn, steps=2000)
# Define the test inputs
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
                                                       x={"x": np.array(test_set.data)},
                                                       y=np.array(test_set.target),
                                                       num_epochs=1,
                                                       shuffle=False)
# Evaluate accuracy.
    accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
    print("\nTest Accuracy: {0:f}\n".format(accuracy_score))
# Classify two new flower samples.
    new_samples = np.array(
                           [[6.4, 3.2, 4.5, 1.5],
                            [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
                                                          x={"x": new_samples},
                                                          num_epochs=1,
                                                          shuffle=False)
    predictions = list(classifier.predict(input_fn=predict_input_fn))
    predicted_classes = [p["classes"] for p in predictions]
    print("New Samples, Class Predictions:  {}\n".format(predicted_classes))
if __name__ == "__main__":
    main()

# MNIST Image data processing

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [ ]:
print("x_train shape = {0}".format(x_train.shape))
print("y_train shape = {0}".format(y_train.shape))
print("x_test shape = {0}".format(x_test.shape))
print("y_test shape = {0}".format(y_test.shape))

In [ ]:
# Let us graph one of the images from MNIST dataset
offset = 0
X1 = x_train[offset].reshape([28,28])
plt.gray()
plt.title("Image at offset: {0}".format(offset))
plt.imshow(X1)
plt.show()
print("Label at offset {0} is: {1}".format(offset, y_train[offset]))

In [ ]:
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [ ]:
training_results = [vectorized_result(y) for y in y_train]
training_inputs = [np.reshape(x, (784, 1)) for x in x_train]
training_data = zip(training_inputs, training_results)

In [ ]:
test_results = [vectorized_result(y) for y in y_test]
test_inputs = [np.reshape(x, (784, 1)) for x in x_test]
test_data = zip(test_inputs, test_results)

In [ ]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):
    
    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
    
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
            gradient descent.  The ``training_data`` is a list of tuples
            ``(x, y)`` representing the training inputs and the desired
            outputs.  The other non-optional parameters are
            self-explanatory.  If ``test_data`` is provided then the
            network will be evaluated against the test data after each
            epoch, and partial progress printed out.  This is useful for
            tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                            training_data[k:k+mini_batch_size]
                            for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(
                                                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
            gradient descent using backpropagation to a single mini batch.
            The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
            is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
            gradient for the cost function C_x.  ``nabla_b`` and
            ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
            to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
            # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
            network outputs the correct result. Note that the neural
            network's output is assumed to be the index of whichever
            neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
        
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
            \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
net = Network([784, 30, 10])

net.SGD(train_data, 30, 10, 3.0, test_data=test_data)